In [1]:
import numpy as np

from jax import jit
import jax.numpy as jnp

import klsurprise as kls

# Setting up a mock case for debuging

In [2]:
# Set the seed for reproducibility
np.random.seed(42)

# Constants and dimensions
theta_dim = 2
D_dim = 7

# Gera um vetor de parametros \theta_1 e theta_2
theta_fid_1 = np.array([0.4, 0.1])
theta_fid_2 = np.array([1.3, 0.2])


# generate F(theta) related quantities
F0 = np.random.rand(D_dim)  # 7x1 vector
M = np.random.rand(D_dim, theta_dim)  # 7x2 matrix
# generate covariance matrix for likelihood L2
C = np.random.rand(D_dim, D_dim)  # 7x7 matrix
# generate covariance matrix for posterior 1
Sigma_1 = np.random.rand(theta_dim, theta_dim)  # 2x2 matrix

# Ensure C and Sigma are symmetric and positive-definite
C = np.dot(C, C.T)
Sigma_1 = np.dot(Sigma_1, Sigma_1.T)
invS1 = np.linalg.inv(Sigma_1)

# covariance matrix of posterior 2
# Assuming Gaussianity and a flat prior, we can also derive the equations for Sigma_2. 
# See https://arxiv.org/abs/1402.3593 eq. A17.
invC  = np.linalg.inv(C)
invS2 = np.dot(M.T, np.dot(invC, M))
Sigma_2 = np.linalg.inv(invS2)

# Define the linear function F(theta)
def F(theta):
    return F0 + np.dot(M, theta)

# fiducial data vectors
################# hey there, maybe you should think some more here about putting noise to this vector!
D1_fid = F(theta_fid_1) 
D2_fid = F(theta_fid_2)

print("Fiducial parameters 1 = ", theta_fid_1)
print("Fiducial parameters 2 = ", theta_fid_2)

Fiducial parameters 1 =  [0.4 0.1]
Fiducial parameters 2 =  [1.3 0.2]


In [3]:
def multivariate_gaussian_pdf(theta, mean, cov):
    """
    Calculate the PDF of a multivariate Gaussian distribution.

    Parameters:
    mean : array-like, shape (n,)
        The mean vector of the Gaussian distribution.
    cov : array-like, shape (n, n)
        The covariance matrix of the Gaussian distribution.
    theta : array-like, shape (n,)
        The parameter vector at which to evaluate the PDF.

    Returns:
    pdf_value : float
        The PDF value of the multivariate Gaussian at the data point.
    """
    k = mean.shape[0]
    diff = theta - mean
    inv_cov = jnp.linalg.inv(cov)
    logL = -0.5 * jnp.dot(diff, jnp.dot(inv_cov, diff))
    norm_factor = jnp.log(jnp.sqrt((2 * jnp.pi) ** k * jnp.linalg.det(cov)))
    logpdf_value = logL - norm_factor
    return logpdf_value

**Define both loglikelihoods that will be used in the main function**

In [4]:
# Define the distributions as callable functions of theta and D
@jit
def logL1(theta):
    return multivariate_gaussian_pdf(theta, theta_fid_1, Sigma_1)

@jit
def logL2(theta):
    return multivariate_gaussian_pdf(theta, theta_fid_2, Sigma_2)

domain = np.array([[5,-5], [5,-5]])

# Debuging part

**Class debug and nested sampling run:**

In [5]:
logL1(np.array([0.4,2]))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Array(-4.2779007, dtype=float32)

In [11]:
sup = kls.surprise_statistics(logL1, F, covariance_matrix_2=Sigma_2, domain=domain, data_2=D2_fid)

In [10]:
res_1 = sup.run_nested_sampling(logL1, print_progress=True)
res_2 = sup.run_nested_sampling(logL2, print_progress=True)

0it [00:00, ?it/s]/home/prm/anaconda3/envs/surprise/lib/python3.10/site-packages/dynesty/dynamicsampler.py:453: RuntimeWarning: overflow encountered in cast
  cur_live_logl[not_finite] = _LOWL_VAL
20300it [00:18, 1055.64it/s, batch: 21 | bound: 2 | nc: 3 | ncall: 99844 | eff(%): 20.228 | loglstar: -10.798 < -1.907 < -1.867 | logz: -4.651 +/-  0.036 | stop:  1.194]        /home/prm/anaconda3/envs/surprise/lib/python3.10/site-packages/dynesty/dynamicsampler.py:453: RuntimeWarning: overflow encountered in cast
  cur_live_logl[not_finite] = _LOWL_VAL
23774it [00:23, 1029.45it/s, batch: 26 | bound: 2 | nc: 1 | ncall: 119259 | eff(%): 19.601 | loglstar: -3.791 < -1.385 < -1.871 | logz: -4.644 +/-  0.034 | stop:  0.977] 
0it [00:00, ?it/s]/home/prm/anaconda3/envs/surprise/lib/python3.10/site-packages/dynesty/dynamicsampler.py:453: RuntimeWarning: overflow encountered in cast
  cur_live_logl[not_finite] = _LOWL_VAL
25130it [00:29, 859.33it/s, batch: 26 | bound: 4 | nc: 1 | ncall: 166231 | eff(

In [14]:
KLD_value = sup.KLD_numerical(res_2, logL2, res_1, logL1)
print("KLD(p2|p1) = {:.2f} nats".format(KLD_value))

Processing batches:   0%|          | 0/26 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/26 [00:00<?, ?it/s]

KLD(p2|p1) = 2.31 nats
